In [6]:
library(rlang)
library(lobstr)

In [7]:
# Define an expression and calculate the result
z <- rlang::expr(y <- x * 10)
x <- 4
eval(z)
y

[1] 40

In [8]:
# Define a multiline expression
a <- {
  c <- 2
  rlang::expr(b <- c * 10)
}
eval(a)
b

[1] 20

In [21]:
# Define an R expression
ast(a <- b + c)

█─`<-` 
├─a 
└─█─`+` 
  ├─b 
  └─c 

In [28]:
# Constants
is_syntactic_literal(x)

[1] TRUE

In [29]:
# Symbols
is.name(x)

[1] FALSE

In [33]:
# Calls: represents a captured function call
is.call(expr(read.table("")))

[1] TRUE

In [49]:
exp1 <- parse(text = c("
x <- 4
y <- 7
"))
exp1[[1]]

[1] "language"

In [60]:
expr_type <- function(x) {
  if (rlang::is_syntactic_literal(x)) {
    "constant"
  } else if (is.symbol(x)) {
    "symbol"
  } else if (is.call(x)) {
    "call"
  } else if (is.pairlist(x)) {
    "pairlist"
  } else {
    typeof(x)
  }
}

In [61]:
switch_expr <- function(x, ...) {
  switch(expr_type(x),
    ...,
    stop("Don't know how to handle type", typeof(x), call. = FALSE)
  )
}

In [71]:
flat_map_chr <- function(.x, .f, ...) {
  purrr::flatten_chr(purrr::map(.x, .f, ...))
}

In [72]:
find_assign_rec <- function(x) {
  switch_expr(x,
    # Base cases
    constant = ,
    symbol = character(),

    # Recursive cases
    pairlist = flat_map_chr(as.list(x), find_assign_rec),
    call = {
      if (is_call(x, "<-")) {
        as_string(x[[2]])
      } else {
        flat_map_chr(as.list(x), find_assign_rec)
      }
    }
  )
}

In [74]:
find_assign <- function(x) unique(find_assign_rec(enexpr(x)))

find_assign({
  a <- 1
  a <- 2
})

ERROR: Error in loadNamespace(i, c(lib.loc, .libPaths()), versionCheck = vI[[i]]): namespace ‘vctrs’ 0.4.2 is already loaded, but >= 0.5.0 is required


In [ ]:
data(diamonds, package = "ggplot2")

# Filter the dataset to include only diamonds with a price greater than 10,000
diamonds_filtered <- diamonds %>%
  filter(price > 10000)